In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import re
import numpy as np
import pandas as pd
import pickle
from mecab import MeCab

In [32]:
#Hyperparameter

hidden_size = 256
PAD_TOKEN = 0
SOS_TOKEN = 1
EOS_TOKEN = 2
UNK_TOKEN = 3
MAX_LENGTH = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [33]:
def clean_text(text):
    if pd.isna(text): # NaN 값 처리
        return ''
    text = text.lower()
    text = re.sub(r'\d+', ' ', text) # 숫자는 공백으로 처리
    text = re.sub(r'([^\w\s])', r' \1 ', text) # 마침표 앞 뒤로 공백 추가
    text = re.sub(r'\s+', ' ', text) # 두 개 이상의 공백을 하나로 처리
    text = text.strip() # 텍스트 양 옆의 공백 제거
    
    return text

In [34]:
def indiceFromSentence(vocab, sentence):
    return [vocab.get(word, vocab['<UNK>']) for word in sentence.split(' ')]

In [35]:
def tensorFromSentence(vocab, sentence):
    indice = indiceFromSentence(vocab, sentence)
    indice.append(EOS_TOKEN)
    return torch.tensor(indice, dtype=torch.long, device=device).view(-1, 1)

In [36]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=2)
        
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden
    
    def initHidden(self):
        return(torch.zeros(2, 1, self.hidden_size, device=device), torch.zeros(2, 1, self.hidden_size, device=device))

In [37]:
class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=2)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.out(output[0])
        return output, hidden
    
    def initHidden(self):
        return(torch.zeros(2, 1, self.hidden_size, device=device), torch.zeros(2, 1, self.hidden_size, device=device))

In [38]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_hidden = encoder.initHidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    loss = 0
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        
    decoder_input = torch.tensor([[SOS_TOKEN]], device=device)
    decoder_hidden = encoder_hidden
    
    # t_r = 0.5
    sentence = []
    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        # if random.random() < t_r:
        topv, topi = decoder_output.topk(1)
        # else:
            # topi = target_tensor[di]
        decoder_input = topi.squeeze().detach()
        loss += criterion(decoder_output, target_tensor[di])
        
        if decoder_input.item() == EOS_TOKEN:
            break
        sentence.append(idx_to_word[decoder_input.item()])
    print(sentence)    
    loss.backward() # 역전파 
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item() / target_length
    
    

In [39]:
def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.01):
    print_loss_total = 0
    
    for iter in range(1, n_iters+1):
        training_pair = random.choice(pairs) # input - target pair
        input_tensor = tensorFromSentence(word_to_idx, training_pair[0]).to(device)
        target_tensor = tensorFromSentence(word_to_idx, training_pair[1]).to(device)
        
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        
        if iter % print_every == 0:
            print_lost_avg = print_loss_total / print_every
            print(f'Iteration : {iter}, Loss : {print_lost_avg: .4f}')
            print_loss_total = 0

In [40]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(word_to_idx, sentence).to(device)
        input_length = input_tensor.size(0)
        encoder_hidden = encoder.initHidden()
        encoder_hidden = tuple([e.to(device) for e in encoder_hidden])
        
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder_hidden(input_tensor[ei], encoder_hidden)
            
        decoder_input = torch.tensor([[SOS_TOKEN]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = [] # output sentence
        
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_TOKEN:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(idx_to_word[topi.item()]) # 최종 아웃풋의 index
            
            decoder_input = topi.squeeze().detach()
        print(' '.join(decoded_words))
        return ' '.join(decoded_words)

In [41]:
def chat(encoder, decoder, max_length=MAX_LENGTH):
    print("Let's chat (type 'bye' to exit)")
    while True:
        input_sentence = input(">>")
        if input_sentence == 'bye':
            break
        output_sentence = evaluate(encoder, decoder, input_sentence)
        print('<', output_sentence)

In [42]:
# load data and preprocessing
df = pd.read_csv('./dementia_fix.csv', sep=',', names=['Question', 'Intention', 'Answer'], skiprows=1)
df['Encoder Inputs'] = df['Question'].apply(clean_text)
df['Decoder Inputs'] = df['Answer'].apply(clean_text)


In [43]:
df['Encoder Inputs']

0       알츠하이머병의 원인으로 매일 소주를 섭취하는 것이 언급되고 있는데 , 이에 대한 근...
1                         알츠하이머병이라는 질병은 유전적 영향을 받는 것인가요 ?
2                     알츠하이머병의 발생 원인에 대한 연구나 발견이 진행 중인가요 ?
3              알츠하이머병의 발병과 관련하여 뇌의 노화로 인한 증상과 원인을 알려주세요 .
4                   알츠하이머병의 원인과 관련된 연구 결과가 있을까요 ? 알려주세요 .
                              ...                        
6618                         치매 치료에는 어떤 운동이나 작업이 효과적일까요 ?
6619    치매 치료의 결과와 과정을 상세히 설명해주세요 . 치매 치료의 효과는 어떻게 나타날...
6620                      치매를 치료하기 위해 어떤 치료 방법들이 효과적일까요 ?
6621                        치매 치료를 위해 어떤 약물이 사용될 수 있을까요 ?
6622                         치매 치료를 위해 어떤 전문가와 협력해야 할까요 ?
Name: Encoder Inputs, Length: 6623, dtype: object

In [44]:
df['Decoder Inputs']

0       알츠하이머병의 정확한 원인은 아직 밝혀지지 않았지만 , 연구들이 알츠하이머병의 발병...
1       알츠하이머병은 현재까지 완전한 원인이 밝혀지지 않았습니다 . 알츠하이머병은 아직 완...
2       알츠하이머병은 치매를 일으키는 가장 흔한 퇴행성 뇌질환으로 , 년 독일 의사 알로이...
3       알츠하이머병은 현재까지 그 발병 원인에 대한 완벽한 해명은 아직 이루어지지 않았습니...
4       알츠하이머병은 복잡한 질환으로 , 아직도 원인이 완전히 밝혀진 것은 아닙니다 . 그...
                              ...                        
6618    치매는 노인들에게 주로 발생하는 뇌질환으로 , 원인과 치료 방법은 아직 완전히 밝혀...
6619    치매는 일상 생활을 수행하는 능력을 심각하게 손상시키는 질환으로 , 후천성 치매와 ...
6620    치매는 노화로 인해 기억력과 지능을 점차적으로 잃는 질병으로 , 알츠하이머병이 주요...
6621    알츠하이머병은 뇌에 변화가 생겨서 인지 기능에 장애가 생기는 신경퇴행성 질환입니다 ...
6622    치매는 현재까지 완전한 치료가 불가능한 치매입니다 . 치매는 다양한 원인에 의해 발...
Name: Decoder Inputs, Length: 6623, dtype: object

In [45]:
input_sentence = [sentence for sentence in df['Encoder Inputs']]
output_sentence = [sentence + "<EOS>" for sentence in df['Decoder Inputs']]

In [46]:
input_sentence[0:5]

['알츠하이머병의 원인으로 매일 소주를 섭취하는 것이 언급되고 있는데 , 이에 대한 근거가 있는지 알려주세요 .',
 '알츠하이머병이라는 질병은 유전적 영향을 받는 것인가요 ?',
 '알츠하이머병의 발생 원인에 대한 연구나 발견이 진행 중인가요 ?',
 '알츠하이머병의 발병과 관련하여 뇌의 노화로 인한 증상과 원인을 알려주세요 .',
 '알츠하이머병의 원인과 관련된 연구 결과가 있을까요 ? 알려주세요 .']

In [47]:
output_sentence[0:5]

['알츠하이머병의 정확한 원인은 아직 밝혀지지 않았지만 , 연구들이 알츠하이머병의 발병 기전에 대해 논의하고 있습니다 . 일부 연구에 따르면 , 유전적인 요소와 뇌의 기능 손상이 관련되어 있다고 알려져 있습니다 . 알츠하이머병은 아밀로이드 베타 단백질과 타우 단백질의 과도한 생성 , 뇌 세포의 비정상적인 활동 , 뇌 조직의 변화로 인해 발생하는 것으로 생각되고 있습니다 . 이러한 변화가 알츠하이머병의 발병 위험을 증가시키고 , 병의 진행을 가속화시킨다는 것입니다 . 알츠하이머병의 발병과 관련된 위험 요소에 대해서는 더 많은 연구와 조사가 필요합니다 . 더 많은 연구와 자료 수집을 통해 알츠하이머병에 대한 더 많은 이해와 예방 방법이 개발될 것으로 기대됩니다 .<EOS>',
 '알츠하이머병은 현재까지 완전한 원인이 밝혀지지 않았습니다 . 알츠하이머병은 아직 완전히 이해되지 않았지만 , 연구 결과에 따르면 유전적인 요소와 다양한 환경적인 요인이 이 질환을 일으키는 역할을 한다고 알려져 있습니다 . 특히 , 아밀로이드 베타 단백질의 비정상적인 축적이 알츠하이머병과 관련이 있는 것으로 알려져 있습니다 . 이 외에도 나이 , 노화 , 고혈압 , 당뇨병 , 그리고 흡연 등과 같은 다른 요인들도 알츠하이머병 발병과 연관성이 있을 수 있습니다 . 더 많은 연구와 조사를 통해 알츠하이머병의 원인을 파악하고 예방 방법을 개발할 필요가 있습니다 .<EOS>',
 '알츠하이머병은 치매를 일으키는 가장 흔한 퇴행성 뇌질환으로 , 년 독일 의사 알로이스 알츠하이머에 의해 처음으로 보고되었습니다 . 이 질환의 원인에 대해서는 현재까지 명확한 답은 없으나 , 치매 발생의 위험 요소와 관련하여 몇 가지 위험 요인이 알려져 있습니다 . 일반적으로 , 가장 잘 알려진 요인 중 하나는 고령입니다 . 고령은 치매의 발병 위험을 증가시키는 가장 큰 위험 요소로 알려져 있습니다 . 또한 , 가족력이 있는 경우 알츠하이머병 발생 위험이 높아집니다 . 연구에 따르면 , 조발성 가족성 알츠하이머병은 주로 

In [48]:
# 단어 사전 생성
all_word = set(' '.join(df['Encoder Inputs'].tolist() + df['Decoder Inputs'].tolist()).split())
vocab = {'<PAD>': PAD_TOKEN, '<SOS>': SOS_TOKEN, '<EOS>': EOS_TOKEN, '<UNK>': UNK_TOKEN}
vocab.update({word: i+4 for i, word in enumerate(all_word)})
vocab_size = len(vocab)

with open('vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

In [49]:
all_word

{'증가가',
 '가지며',
 '암의',
 '활동으로서',
 '입히므로',
 '리스페리돈',
 '행복하게',
 '중요성을',
 '마약',
 '수정하기',
 '그룹치료',
 '상실하여',
 '뇌동맥류',
 '가져오는',
 '의미로',
 '일들을',
 '인지재활치료나',
 '컴퓨터단층촬영',
 'behavioral',
 '진단받는',
 '화학물질은',
 '나뉘며',
 '진단해주는',
 '체중감소',
 '알츠하이머병에서',
 '혼합하여',
 '과정이며',
 '검진기관은',
 '주치의의',
 '사고도',
 '겪기도',
 '방법이라고',
 '알약이',
 '프로락틴',
 '근육의',
 '연결될',
 '독소',
 '수준일',
 '병행하도록',
 '변형으로',
 '고령자들의',
 '줄어든다는',
 '효용성과',
 '노출된',
 '생깁니다',
 '경각심을',
 '사는',
 '치매에서는',
 '조언이나',
 '예방적인',
 '결합으로',
 '흡연자는',
 '고령화는',
 '않는다면',
 '일어나고',
 '받으려면',
 '쌓이지',
 '식사하지',
 '측면인',
 '변경될',
 '이론에',
 '방식들은',
 '의심해볼',
 '우울함의',
 '무산소',
 '안전에',
 '포함되나요',
 '있었습니다',
 '유병률은',
 '염증성',
 '장기적이고',
 '모호해지며',
 '생기는',
 '모임',
 '역학',
 '자유로워질',
 '것인지에',
 '유지하여',
 '못할',
 '개인차와',
 '요인이',
 '치매이며',
 '부작용에는',
 '확인되지',
 '혈류가',
 '개선시킵니다',
 '발생하게',
 '행동이므로',
 '한꺼번에',
 '좋아지고',
 '시각',
 '발병률을',
 '앞장서야',
 'induced',
 '넘치며',
 '축적됨으로써',
 '콜린성약제는',
 '생각하지만',
 '됩니다',
 '계획성',
 '학습할',
 '이미프라민과',
 '강화하세요',
 '고유한',
 '기간',
 '단점이',
 '동반으로',
 '대화의',
 '혈액량을',
 '속한다',
 '문진과',
 '관계

In [50]:
vocab

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 '<UNK>': 3,
 '증가가': 4,
 '가지며': 5,
 '암의': 6,
 '활동으로서': 7,
 '입히므로': 8,
 '리스페리돈': 9,
 '행복하게': 10,
 '중요성을': 11,
 '마약': 12,
 '수정하기': 13,
 '그룹치료': 14,
 '상실하여': 15,
 '뇌동맥류': 16,
 '가져오는': 17,
 '의미로': 18,
 '일들을': 19,
 '인지재활치료나': 20,
 '컴퓨터단층촬영': 21,
 'behavioral': 22,
 '진단받는': 23,
 '화학물질은': 24,
 '나뉘며': 25,
 '진단해주는': 26,
 '체중감소': 27,
 '알츠하이머병에서': 28,
 '혼합하여': 29,
 '과정이며': 30,
 '검진기관은': 31,
 '주치의의': 32,
 '사고도': 33,
 '겪기도': 34,
 '방법이라고': 35,
 '알약이': 36,
 '프로락틴': 37,
 '근육의': 38,
 '연결될': 39,
 '독소': 40,
 '수준일': 41,
 '병행하도록': 42,
 '변형으로': 43,
 '고령자들의': 44,
 '줄어든다는': 45,
 '효용성과': 46,
 '노출된': 47,
 '생깁니다': 48,
 '경각심을': 49,
 '사는': 50,
 '치매에서는': 51,
 '조언이나': 52,
 '예방적인': 53,
 '결합으로': 54,
 '흡연자는': 55,
 '고령화는': 56,
 '않는다면': 57,
 '일어나고': 58,
 '받으려면': 59,
 '쌓이지': 60,
 '식사하지': 61,
 '측면인': 62,
 '변경될': 63,
 '이론에': 64,
 '방식들은': 65,
 '의심해볼': 66,
 '우울함의': 67,
 '무산소': 68,
 '안전에': 69,
 '포함되나요': 70,
 '있었습니다': 71,
 '유병률은': 72,
 '염증성': 73,
 '장기적이고': 74,
 '모호해지며': 75,
 '생기는': 76,
 '모

In [51]:
word_to_idx = vocab
idx_to_word = {i: word for word, i in word_to_idx.items()}

In [52]:
word_to_idx

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 '<UNK>': 3,
 '증가가': 4,
 '가지며': 5,
 '암의': 6,
 '활동으로서': 7,
 '입히므로': 8,
 '리스페리돈': 9,
 '행복하게': 10,
 '중요성을': 11,
 '마약': 12,
 '수정하기': 13,
 '그룹치료': 14,
 '상실하여': 15,
 '뇌동맥류': 16,
 '가져오는': 17,
 '의미로': 18,
 '일들을': 19,
 '인지재활치료나': 20,
 '컴퓨터단층촬영': 21,
 'behavioral': 22,
 '진단받는': 23,
 '화학물질은': 24,
 '나뉘며': 25,
 '진단해주는': 26,
 '체중감소': 27,
 '알츠하이머병에서': 28,
 '혼합하여': 29,
 '과정이며': 30,
 '검진기관은': 31,
 '주치의의': 32,
 '사고도': 33,
 '겪기도': 34,
 '방법이라고': 35,
 '알약이': 36,
 '프로락틴': 37,
 '근육의': 38,
 '연결될': 39,
 '독소': 40,
 '수준일': 41,
 '병행하도록': 42,
 '변형으로': 43,
 '고령자들의': 44,
 '줄어든다는': 45,
 '효용성과': 46,
 '노출된': 47,
 '생깁니다': 48,
 '경각심을': 49,
 '사는': 50,
 '치매에서는': 51,
 '조언이나': 52,
 '예방적인': 53,
 '결합으로': 54,
 '흡연자는': 55,
 '고령화는': 56,
 '않는다면': 57,
 '일어나고': 58,
 '받으려면': 59,
 '쌓이지': 60,
 '식사하지': 61,
 '측면인': 62,
 '변경될': 63,
 '이론에': 64,
 '방식들은': 65,
 '의심해볼': 66,
 '우울함의': 67,
 '무산소': 68,
 '안전에': 69,
 '포함되나요': 70,
 '있었습니다': 71,
 '유병률은': 72,
 '염증성': 73,
 '장기적이고': 74,
 '모호해지며': 75,
 '생기는': 76,
 '모

In [53]:
idx_to_word

{0: '<PAD>',
 1: '<SOS>',
 2: '<EOS>',
 3: '<UNK>',
 4: '증가가',
 5: '가지며',
 6: '암의',
 7: '활동으로서',
 8: '입히므로',
 9: '리스페리돈',
 10: '행복하게',
 11: '중요성을',
 12: '마약',
 13: '수정하기',
 14: '그룹치료',
 15: '상실하여',
 16: '뇌동맥류',
 17: '가져오는',
 18: '의미로',
 19: '일들을',
 20: '인지재활치료나',
 21: '컴퓨터단층촬영',
 22: 'behavioral',
 23: '진단받는',
 24: '화학물질은',
 25: '나뉘며',
 26: '진단해주는',
 27: '체중감소',
 28: '알츠하이머병에서',
 29: '혼합하여',
 30: '과정이며',
 31: '검진기관은',
 32: '주치의의',
 33: '사고도',
 34: '겪기도',
 35: '방법이라고',
 36: '알약이',
 37: '프로락틴',
 38: '근육의',
 39: '연결될',
 40: '독소',
 41: '수준일',
 42: '병행하도록',
 43: '변형으로',
 44: '고령자들의',
 45: '줄어든다는',
 46: '효용성과',
 47: '노출된',
 48: '생깁니다',
 49: '경각심을',
 50: '사는',
 51: '치매에서는',
 52: '조언이나',
 53: '예방적인',
 54: '결합으로',
 55: '흡연자는',
 56: '고령화는',
 57: '않는다면',
 58: '일어나고',
 59: '받으려면',
 60: '쌓이지',
 61: '식사하지',
 62: '측면인',
 63: '변경될',
 64: '이론에',
 65: '방식들은',
 66: '의심해볼',
 67: '우울함의',
 68: '무산소',
 69: '안전에',
 70: '포함되나요',
 71: '있었습니다',
 72: '유병률은',
 73: '염증성',
 74: '장기적이고',
 75: '모호해지며',
 76: '생기는',
 77

In [54]:
word_to_idx['이루고']

7808

In [55]:
idx_to_word[20]

'인지재활치료나'

In [56]:
encoder = EncoderLSTM(vocab_size, hidden_size).to(device)
decoder = DecoderLSTM(hidden_size, vocab_size).to(device)

In [57]:
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.005)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()

In [58]:
pairs = [list(x) for x in zip(df['Encoder Inputs'], df['Decoder Inputs'])]

In [59]:
pairs[1]

['알츠하이머병이라는 질병은 유전적 영향을 받는 것인가요 ?',
 '알츠하이머병은 현재까지 완전한 원인이 밝혀지지 않았습니다 . 알츠하이머병은 아직 완전히 이해되지 않았지만 , 연구 결과에 따르면 유전적인 요소와 다양한 환경적인 요인이 이 질환을 일으키는 역할을 한다고 알려져 있습니다 . 특히 , 아밀로이드 베타 단백질의 비정상적인 축적이 알츠하이머병과 관련이 있는 것으로 알려져 있습니다 . 이 외에도 나이 , 노화 , 고혈압 , 당뇨병 , 그리고 흡연 등과 같은 다른 요인들도 알츠하이머병 발병과 연관성이 있을 수 있습니다 . 더 많은 연구와 조사를 통해 알츠하이머병의 원인을 파악하고 예방 방법을 개발할 필요가 있습니다 .']

In [60]:
trainIters(encoder, decoder, 10000, 100)

['전류를', '관련되었는지도', '제기되고', '축적이라는', '평가되고', '관리법에', '관련되었는지도', '관련되었는지도', '알려진', '알려진', '알려진', '알려진', '알려진', '저해하거나', '실행기능', '땀이', '목표는', '재처방이', '재처방이', '관련되었는지도', '평가되고', '평가되고', '관리법에', '관련되었는지도', '평가되고', '알려진', '알려진', '알려진', '알려진', '저해하거나', '땀이', '실행기능', '땀이', '재처방이', '루틴에', '재처방이', '루틴에', '재처방이', '루틴에', '루틴에', '재처방이', '루틴에', '재처방이', '루틴에', '평가되고', '평가되고', '추천되어야', '추천되어야', '영양소는', '영양소는', '루틴에', '영양소는', '루틴에', '영양소는', '루틴에', '주제의', '영양소는', '영양소는', '의심되면', '영양소는', '의심되면', '말초신경', '말초신경', '말초신경', '재처방이', '재처방이', '평가되고', '평가되고', '관리법에', '평가되고', '관리법에', '물건들을', '관련되었는지도', '평가되고', '평가되고', '평가되고', '관리법에', '관련되었는지도', '평가되고', '알려진', '알려진', '알려진', '알려진', '저해하거나', '땀이', '목표는', '시행되나요']
['치매는', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '

KeyboardInterrupt: 

In [ ]:
encoder.eval()
decoder.eval()